In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
# import optuna
import time
import sys
import torch
import os
import argparse
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
from load_data_funs import load_data, gen_batch_data_fixations_choice, gen_batch_data_fixations_only, gen_batch_data_choice_only
from train_and_test_funs import test, train_on_simulation_then_human_with_intermediate_tests, test_record_each_output, compute_heldout_performance
from neural_nets import SimpleLSTM, SimpleMLP, SimpleGRU, SimpleTransformer

In [6]:
model_name = 'MLP'
train_seq_part = 'choice_only'
n_simulation_sequences_train = 1e3
n_simulation_sequences_train = 1e3
n_human_sequences_train = 0
n_sequences_test = 500
n_sequences_final_performance = 500
d_model = 128
n_layers = 2
n_head = 2
sim_lr = .001
human_lr = .001
batch_size = 32
dropout = 0
run_idx = 0
on_cluster = True
test_batch_increment_sim = 200
test_batch_increment_human = 200 

In [8]:
print(type(n_simulation_sequences_train))

####################################################
#### Set cluster folders
#############################################
# set path to the human and simulation data
if on_cluster:
    sim_data_path = '/scratch/gpfs/erussek/RNN_project/optimal_fixation_sims'
    human_data_path = '/scratch/gpfs/erussek/RNN_project/human_trials.json'
else:
    sim_data_path = '/Users/erussek/Dropbox/Griffiths_Lab_Stuff/Data/RNNs/optimal_fixation_sims'
    human_data_path = '/Users/erussek/Dropbox/Griffiths_Lab_Stuff/Data/RNNs/human_trials.json'



<class 'float'>


In [9]:
# so, for choice only - want the input to just have chosen vs unchosen... 
train_data_sim, val_data_sim, test_data_sim, train_data_human, val_data_human, test_data_human = load_data(sim_data_path, human_data_path,this_seed=run_idx,split_human_data=True)


In [11]:
batch_size=32
batch_idx = 3
data = train_data_human
use_human_data = True

In [14]:
batch_sim_data = data[batch_idx*batch_size:((batch_idx+1)*(batch_size))]
batch_choices_idx = [trial_data['choice'] - 1 for trial_data in batch_sim_data]


In [15]:
batch_choices_idx

[1,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 2,
 1,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 2,
 1,
 1,
 0,
 2]

In [105]:
class SimpleChoiceOnly(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input_output = nn.Linear(1,1)

    def forward(self, x):
        
        
        o = torch.zeros(x.shape)
        for item in range(3):
            
            a = torch.zeros(x.shape[0],1)
            a[:,0] = x[:,item]
            o[:,item] = torch.squeeze(input_output(a))

        return o


In [102]:
    # Functions to generate data
    gen_data_func_by_train_seq_part = {"fix_and_choice": gen_batch_data_fixations_choice, "fix_only": gen_batch_data_fixations_only, "choice_only": gen_batch_data_choice_only, "choice_then_fix": gen_batch_data_choice_only}
    gen_data_func_pre = gen_data_func_by_train_seq_part[train_seq_part]
    # set the fix unit
    gen_data_func = lambda a, b, c, use_human_data=False : gen_data_func_pre(a, b, c, fix_unit = fix_unit, use_human_data = use_human_data)
    
    fix_unit = 'ID'

In [103]:
model = SimpleChoiceOnly()
    # non neural net training parameters
criterion   = torch.nn.MSELoss()
start_time = time.time()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
simulation_loss_results, human_loss_results, train_sequence_number,human_sequence_number, simulation_sequence_number, model = train_on_simulation_then_human_with_intermediate_tests(model,train_data_sim, train_data_human,val_data_sim,val_data_human,criterion,device,batch_size,n_simulation_sequences_train, n_human_sequences_train, n_sequences_test, gen_data_func, sim_lr = sim_lr, human_lr = human_lr, test_batch_increment_sim=test_batch_increment_sim, test_batch_increment_human=test_batch_increment_human)


cpu
Training on simulated data
Training on human data


In [47]:
n_total_seq = 100
model.eval()

n_batches = int(np.round(n_total_seq / batch_size));

loss_res = np.zeros((n_batches, 1), dtype=float)

In [106]:
model = SimpleChoiceOnly()

test_data = test_data_human
gen_batch_data = gen_data_func
batch_idx = 3
data, target = gen_batch_data(batch_size, batch_idx, test_data, use_human_data=use_human_data)
data, target = torch.from_numpy(data).float().to(device), torch.from_numpy(target).long().to(device)
model(data)

tensor([[-0.1103,  0.7463,  0.7463],
        [ 0.7463,  0.7463, -0.1103],
        [ 0.7463, -0.1103,  0.7463],
        [ 0.7463,  0.7463, -0.1103],
        [-0.1103,  0.7463,  0.7463],
        [-0.1103,  0.7463,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [-0.1103,  0.7463,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [-0.1103,  0.7463,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [-0.1103,  0.7463,  0.7463],
        [ 0.7463,  0.7463, -0.1103],
        [ 0.7463,  0.7463, -0.1103],
        [-0.1103,  0.7463,  0.7463],
        [ 0.7463, -0.1103,  0.7463],
        [ 0.7463,  0.7463, -0.1103],
        [ 0.7463,  0.7463, -0.1103],
        [ 0.7463,  0.7463, -0.1103],
        [-0.1103,  0.7463,  0.7463],
        [ 0.7463,  0.7463, -0.1103],
        [ 0.7463, -0.1103,  0.7463],
        [ 0.7463,  0.7463, -0.1103],
        [ 0.7463, -0.1103,  0.7463],
 

In [67]:
torch.transpose(data[:,0], 0, 0)

tensor([1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [99]:
input_output = nn.Linear(1,1)
x = data
o = torch.zeros(x.shape)
for item in range(3):
    a = torch.zeros(x.shape[0],1)
    a[:,0] = x[:,0]
    o[:,item] = torch.squeeze(input_output(a))

In [98]:
torch.squeeze(input_output(a))

tensor([0.5630, 0.7230, 0.7230, 0.7230, 0.5630, 0.5630, 0.7230, 0.5630, 0.7230,
        0.5630, 0.7230, 0.7230, 0.7230, 0.7230, 0.5630, 0.7230, 0.7230, 0.5630,
        0.7230, 0.7230, 0.7230, 0.7230, 0.5630, 0.7230, 0.7230, 0.7230, 0.7230,
        0.7230, 0.7230, 0.7230, 0.7230, 0.7230], grad_fn=<SqueezeBackward0>)

In [ ]:
        o = torch.zeros(x.shape)
        for item in range(3):
            o[:,item] = self.input_output(x[:,item].transpose())

In [77]:
torch.transpose(x[:,item],-1,-1).shape

torch.Size([32])

In [88]:
x.shape[0]

32

In [83]:
a = torch.zeros(x.shape[0],1)
a[:,0] = x[:,0]

In [86]:
input_output(a)

tensor([[ 0.1322],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [ 0.1322],
        [ 0.1322],
        [-0.4945],
        [ 0.1322],
        [-0.4945],
        [ 0.1322],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [ 0.1322],
        [-0.4945],
        [-0.4945],
        [ 0.1322],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [ 0.1322],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945],
        [-0.4945]], grad_fn=<AddmmBackward0>)